In [3]:
import numpy as np, pandas as pd
import seaborn as sns, matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [4]:
data=pd.read_csv("/Users/rebeccaharrison/Documents/Python/Bootcamp/Data/news_articles.csv")
data.head()

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0


In [5]:

# some few bits of missing information - so few prob can drop
data=data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2045 entries, 0 to 2045
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   author                   2045 non-null   object 
 1   published                2045 non-null   object 
 2   title                    2045 non-null   object 
 3   text                     2045 non-null   object 
 4   language                 2045 non-null   object 
 5   site_url                 2045 non-null   object 
 6   main_img_url             2045 non-null   object 
 7   type                     2045 non-null   object 
 8   label                    2045 non-null   object 
 9   title_without_stopwords  2045 non-null   object 
 10  text_without_stopwords   2045 non-null   object 
 11  hasImage                 2045 non-null   float64
dtypes: float64(1), object(11)
memory usage: 207.7+ KB


In [6]:
# Creating binary outcome variable
data.loc[data['label'] == 'Fake', 'label'] = 1
data.loc[data['label'] == 'Real', 'label'] = 0


In [7]:
# checking out has image
data['hasImage'].value_counts()

1.0    1580
0.0     465
Name: hasImage, dtype: int64

In [8]:
# checking and cleaning author

data.loc[(data['author'] == '-NO AUTHOR-'), 'author'] = 'No Author'
data['author'].value_counts()[:20]


No Author                              527
Activist Post                           82
EdJenner                                73
noreply@blogger.com (Der Postillon)     72
Anonymous                               58
Daniel Greenfield                       55
admin                                   48
Jason Easley                            45
Alex Ansary                             44
Dr. Patrick Slattery                    38
Henry Wolff                             33
Corbett                                 27
tokyowashi (noreply@blogger.com)        20
Mike Rivero                             17
Steve Watson                            16
Sarah Jones                             16
Hrafnkell Haraldsson                    15
Roosh Valizadeh                         15
Letsbereal                              13
ActivistPost                            12
Name: author, dtype: int64

In [9]:
#set date to date
data['date']=pd.to_datetime(data['published'], infer_datetime_format=True, errors='coerce')


In [10]:
# creating a variable that counts no of posts
data['post_no'] = data.groupby('author').cumcount() + 1
data.loc[data['author'] == 'No Author', 'post_no'] = 0
data[['author','post_no','published']]




,author,post_no,published
0,Barracuda Brigade,1,2016-10-26T21:41:00.000+03:00
1,reasoning with facts,1,2016-10-29T08:47:11.259+03:00
2,Barracuda Brigade,2,2016-10-31T01:41:49.479+02:00
3,Fed Up,1,2016-11-01T05:22:00.000+02:00
4,Fed Up,2,2016-11-01T21:56:00.000+02:00
...,...,...,...
2041,Matt Barber,2,2016-10-27T03:04:50.327+03:00
2042,Jane Chastain,2,2016-10-27T03:04:50.704+03:00
2043,Michael Brown,2,2016-10-27T03:04:54.788+03:00
2044,Ann Coulter,3,2016-10-27T03:05:01.989+03:00


In [12]:
data.head()

data.iloc[:-1,0:3].boxplot(figsize=(10,10),vert=False)
plt.show()

,hasImage,post_no
count,2045.000000,2045.000000
mean,0.772616,9.655746
std,0.419245,16.192573
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,2.000000
75%,1.000000,11.000000
max,1.000000,82.000000
